In [1]:
import pandas as pd
import numpy as np
from time import time

In [2]:
col_names = ["duration","protocol_type","service","flag","src_bytes",
    "dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins",
    "logged_in","num_compromised","root_shell","su_attempted","num_root",
    "num_file_creations","num_shells","num_access_files","num_outbound_cmds",
    "is_host_login","is_guest_login","count","srv_count","serror_rate",
    "srv_serror_rate","rerror_rate","srv_rerror_rate","same_srv_rate",
    "diff_srv_rate","srv_diff_host_rate","dst_host_count","dst_host_srv_count",
    "dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate",
    "dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate",
    "dst_host_rerror_rate","dst_host_srv_rerror_rate","label"]

raw_data = pd.read_csv("./kddcup.data_10_percent_corrected", header=None, names = col_names)
raw_data['label'] = raw_data['label'].replace(['back.', 'buffer_overflow.', 'ftp_write.', 'guess_passwd.', 'imap.', 'ipsweep.', 'land.', 'loadmodule.', 'multihop.', 'neptune.', 'nmap.', 'perl.', 'phf.', 'pod.', 'portsweep.', 'rootkit.', 'satan.', 'smurf.', 'spy.', 'teardrop.', 'warezclient.', 'warezmaster.'], 'attack')

In [3]:
def categorical_converter(data):
    '''manually encode the three categorical columns into ordinals'''
    cols=['protocol_type','service','flag','label']
    for i in cols:
        data[i] = le.fit_transform(data[i])
    data = hot.fit_transform(data.loc[:,cols])

In [4]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
le = LabelEncoder()
hot = OneHotEncoder(sparse=True, handle_unknown='ignore')
categorical_converter(raw_data)
# 1 normal

In [5]:
features= raw_data.iloc[:,:raw_data.shape[1]]
labels= raw_data.iloc[:,raw_data.shape[1]-1:]
labels= labels.values.ravel() # this becomes a 'horizontal' array

In [6]:
from sklearn.model_selection import train_test_split
df= pd.DataFrame(features)
X_train, X_test, y_train, y_test = train_test_split(df, labels, train_size=0.1, test_size=0.9)


In [7]:
X_train_features= X_train.iloc[:,:raw_data.shape[1]-1]


In [8]:
#normalization
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler().fit(X_train_features)
#normalization of training dataset 
X_train.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_train.iloc[:,:raw_data.shape[1]-1])


/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:6: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [9]:
#labels
X_train_transformed = pd.DataFrame(X_train)
X = X_train_transformed[X_train_transformed.label==1]
X1 = X.iloc[:,:41]
X = X_train_transformed[X_train_transformed.label==0]
X0 = X.iloc[:,:41]

In [10]:
from sklearn.cluster import KMeans

t0 = time()
numberofclasses = 50
kmeans1 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X1)
kmeans0 = KMeans(n_clusters=numberofclasses, random_state=0).fit(X0)
tt = time()-t0

print("Classifier trained in {} seconds".format(round(tt,3)))

Classifier trained in 7.869 seconds


In [11]:
label_train = np.append(np.zeros(numberofclasses)+1,np.zeros(numberofclasses))
data_train = np.append(kmeans1.cluster_centers_,kmeans0.cluster_centers_,axis=0)


In [12]:
from sklearn.neighbors import KNeighborsClassifier 

knn = KNeighborsClassifier(n_neighbors=1,weights = 'distance')
knn.fit(data_train, label_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=1, p=2,
           weights='distance')

In [13]:
score=knn.score(X_train.iloc[:,:raw_data.shape[1]-1],X_train.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
score

0.9977936115946723

In [14]:
#normalization of testing dataset 
X_test.iloc[:,:raw_data.shape[1]-1] = scaler.transform(X_test.iloc[:,:raw_data.shape[1]-1]) 

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
/opt/conda/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [15]:
score=knn.score(X_test.iloc[:,:raw_data.shape[1]-1],X_test.iloc[:,raw_data.shape[1]-1:],sample_weight=None)
score

0.9977418868739303